# Interactive choropleth maps  with Python and Folium for Srilanka (District level)

*Choropleth maps provide an easy way to visualize how a measurement varies across a geographic area or show the level of variability within a region. A heat map is similar but does not use geographic boundaries*

In [1]:
#Let's import libraries we need
import pandas as pd
import geopandas as gpd
import folium

In [2]:
# importing geo json files 


disg = gpd.read_file('/kaggle/input/srilanka-geo/District_geo.json')


In [3]:
# loading first two rows of the data set
disg.head(2)

,ADM2_EN,ADM2_SI,ADM2_TA,ADM2_PCODE,ADM2_REF,ADM2ALT1EN,ADM2ALT2EN,ADM2ALT1SI,ADM2ALT2SI,ADM2ALT1TA,...,ADM0_EN,ADM0_SI,ADM0_TA,ADM0_PCODE,Shape_Leng,Shape_Area,date,validOn,validTo,geometry
0,[unknown],[unknown],[unknoபுத்தளம் மாவட்ட,LK60,,,,,,,...,Sri Lanka,ශ්‍රී ලංකා,இலங்கை,LK,0.286741,0.002137,2019-03-04T00:00:00,2020-03-05T00:00:00,1899-11-30T00:00:00,"POLYGON ((79.82493 7.78566, 79.82341 7.78844, ..."
1,Ampara,අම්පාර දිස්ත්‍රික,அம்பாறை மாவட்டம்,LK52,,,,,,,...,Sri Lanka,ශ්‍රී ලංකා,இலங்கை,LK,5.397133,0.366562,2019-03-04T00:00:00,2020-03-05T00:00:00,1899-11-30T00:00:00,"POLYGON ((81.75144 7.45764, 81.74739 7.45469, ..."


In [4]:
#removing other data except "ADM2_PCODE" and "geometry"
disg=disg[['ADM2_PCODE','geometry']]



In [5]:
# loading aditional data that we want to load to final map other than  properties of Geojson file
dis_a=pd.read_csv('/kaggle/input/srilanka-geo/dristrict_aditional_data.csv')
dis_a.rename(columns={'admin2Pcode':'ADM2_PCODE'}, inplace=True)
dis_a.head()

,admin2Name_en,admin2Name_si,ADM2_PCODE,risk_level,patients,population
0,[unknown],[unknown],LK60,1,100,30
1,Ampara,අම්පාර දිස්ත්‍රික්කය,LK52,1,110,33
2,Anuradhapura,අනුරාධපුර දිස්ත්‍රික්කය,LK71,1,121,36
3,Badulla,බදුල්ල දිස්ත්‍රික්කය,LK81,1,133,40
4,Batticaloa,මඩකලපුව දිස්ත්‍රික්කය,LK51,1,146,44


In [6]:
#merging both geo data and adttioanl data into one data fame for generate map mapping key is '"ADM2_PCODE""

district=disg.merge(dis_a,on="ADM2_PCODE")
district.head()

,ADM2_PCODE,geometry,admin2Name_en,admin2Name_si,risk_level,patients,population
0,LK60,"POLYGON ((79.82493 7.78566, 79.82341 7.78844, ...",[unknown],[unknown],1,100,30
1,LK52,"POLYGON ((81.75144 7.45764, 81.74739 7.45469, ...",Ampara,අම්පාර දිස්ත්‍රික්කය,1,110,33
2,LK71,"POLYGON ((80.93922 8.34202, 80.93707 8.34614, ...",Anuradhapura,අනුරාධපුර දිස්ත්‍රික්කය,1,121,36
3,LK81,"POLYGON ((81.26884 7.27764, 81.26596 7.28468, ...",Badulla,බදුල්ල දිස්ත්‍රික්කය,1,133,40
4,LK51,"MULTIPOLYGON (((81.72618 7.63616, 81.72664 7.6...",Batticaloa,මඩකලපුව දිස්ත්‍රික්කය,1,146,44


In [ ]:
# Defining the style function and Highlight function 

style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}



# declaring the map with starting point
m = folium.Map(location=[7.8731, 80.7718], zoom_start=7)

choropleth=folium.Choropleth(
    geo_data=district,
    name='choropleth',
    data= district,
    columns=['ADM2_PCODE', 'patients'],
    key_on='properties.ADM2_PCODE',
    fill_color='PuRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    
    legend_name='Covid-19 risk Level',

).add_to(m)

# adding some geo loacation (location pins )

folium.Marker(
    location=[6.905655, 79.927357],
    popup='Point one',
    
    icon=folium.Icon(icon='cloud')
).add_to(m)

folium.Marker(
    location=[6.7106,79.9074],
    popup='Point two',
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(m)

folium.Marker(
    location=[7.9403,81.01886823],
    popup='Point three',
    icon=folium.Icon(color='blue', icon='info-sign')
).add_to(m)

folium.Marker(
    location=[9.3803, 80.3770],
    popup='Point four',
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(m)

#adding the same geo layer again on top of previouse layer to make interactivity 

INT = folium.features.GeoJson(
    district,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['admin2Name_en','patients','population'],
        aliases=['District: ','Numer of Patients:','Population'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
m.add_child(INT)
m.keep_in_front(INT)
folium.LayerControl().add_to(m)



In [ ]:
#m.save('District_final.html')